In [1]:
import numpy as np
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import csv
from tqdm import tqdm
import json

In [2]:
event_cutoff = 300
small_pull_cutoff = 0.2
raw_data_file = "./data/raw/SheetPullData.csv"
processed_data_file = "./data/processed_sheets_pull.csv"

In [3]:
# helper functions
def clip(x):
    if x >= 0: return x
    return 0

def process_timestamp(timestamp_str):
    # TODO: pad timeframe into standard UNIX time?
    return timestamp_str

def enforce_list_identical(some_list):
    assert len(some_list) > 0
    elem = some_list[0]
    for obj in some_list:
        assert elem == obj, "Mismatch! First element: {} Current element: {}".format(elem, obj)

In [4]:
# Step 1: read dataset while merging events
def prep_feature(event_list_rows):
    '''
    event: a list of rows where each element is a dict with column
            names in SheetPullData.csv as keys. List may or may
            not contain multiple rows.
    
    Return: a pandas.DataFrame object that can be appended
    '''
    
    # Read out all columns in raw dataset
    HH_ID_list = [d['HH_ID'] for d in event_list_rows]
    roll_ID_list = [d['Roll_ID'] for d in event_list_rows]
    roll_Type_list = [d['Roll_Type'] for d in event_list_rows]
    timestamp_list = [process_timestamp(d['Timestamp']) for d in event_list_rows]
    seconds_since_last_list = [float(d['Time_since_last_pull']) for d in event_list_rows]
    sheets_cnt_list = [float(d['Sheets']) for d in event_list_rows]
    
    # Sanity Check
    enforce_list_identical(HH_ID_list)
    enforce_list_identical(roll_ID_list)
    enforce_list_identical(roll_Type_list)
    assert len(HH_ID_list) == len(timestamp_list)
    
    # Preprocess rule 1: clip negative values
    sheets_cnt_list = [clip(i) for i in sheets_cnt_list]
    
    # build features
    cur_row = {
        'seconds_since_previous_pull': [seconds_since_last_list[0]],
        'HH_ID': [HH_ID_list[0]],
        'roll_ID': [roll_ID_list[0]],
        'roll_type': [roll_Type_list[0]],
        'starting_time': [timestamp_list[0]],
        'elapsed_time': [sum(seconds_since_last_list[1:])],
        'total_sheets_used': [sum(sheets_cnt_list)],
        'number_pulled': [len(HH_ID_list)],
        'max_min_pull_diff': [max(sheets_cnt_list) - min(sheets_cnt_list)],
        'multiple_pull_flag': [(len(HH_ID_list) > 1)],
        'average_time_between_pulls': [np.mean(seconds_since_last_list[1:]) if (len(HH_ID_list) > 1) else 0]
    }
    return pd.DataFrame(data = cur_row)

# remove cached variable
try:
    delete(ret_df)
except NameError:
    pass

with open(raw_data_file, newline = '') as csvfile:
    reader = csv.DictReader(csvfile)
    # read to memory
    all_rows = list(reader)
    first_row = all_rows[0]
    event_rows = [first_row]
    for row in tqdm(all_rows):
        if float(row['Time_since_last_pull']) < event_cutoff:
            event_rows.append(row)
        else:
            new_df = prep_feature(event_rows)
            try:
                ret_df = ret_df.append(new_df)
            except NameError:
                ret_df = new_df
            event_rows = [row]
    if event_rows:
        new_df = prep_feature(event_rows)
        ret_df = ret_df.append(new_df)

100%|██████████| 12785/12785 [00:09<00:00, 1407.14it/s]


In [5]:
ret_df

,seconds_since_previous_pull,HH_ID,roll_ID,roll_type,starting_time,elapsed_time,total_sheets_used,number_pulled,max_min_pull_diff,multiple_pull_flag,average_time_between_pulls
0,4282.0,A,1200,Small,2019-10-29 05:51:42,0.0,3.991088,1,0.000000,False,0.00
0,4282.0,A,1200,Small,2019-10-29 05:51:42,0.0,3.991088,1,0.000000,False,0.00
0,2505.0,A,1200,Small,2019-10-29 06:33:27,0.0,0.477180,1,0.000000,False,0.00
0,2394.0,A,1200,Small,2019-10-29 07:13:21,0.0,4.277735,1,0.000000,False,0.00
0,14200.0,A,1200,Small,2019-10-29 11:10:01,0.0,2.363388,1,0.000000,False,0.00
...,...,...,...,...,...,...,...,...,...,...,...
0,1919.0,D,13655,Big,2020-01-13 07:05:46,32.0,16.694831,3,2.366350,True,16.00
0,347.0,D,13655,Big,2020-01-13 07:12:05,0.0,1.861284,1,0.000000,False,0.00
0,29363.0,D,13655,Big,2020-01-13 15:21:28,6.0,9.511939,2,2.990052,True,6.00
0,4364.0,D,13655,Big,2020-01-13 16:34:18,61.0,29.036462,5,11.062394,True,15.25


In [6]:
ret_df = ret_df[ret_df['total_sheets_used'] > 1]

In [7]:
ret_df.to_csv('./data/processed_sheets_pull.csv')